## The Ground Truth Duplicate Problem

Since the test set has overlapping patches from the same tile, and ground truth labels are **tile-based** (not patch-based), then:

### **Scenario: A rock at the boundary between two patches**

```
Tile 2626_1102 (original 2000×2000)
┌─────────────────────────────────┐
│                                 │
│  ┌──────────┐                  │
│  │ Patch    │ 🪨 ← Rock here   │
│  │ 0_0      │                  │
│  └──────────┘                  │
│       ┌──────────┐             │
│       │ Patch    │ 🪨 ← Same rock
│       │ 0_1      │             │
│       └──────────┘             │
│                                 │
└─────────────────────────────────┘
```

**When Alexis created patch labels from tile labels:**
- He took tile-level annotations (point coordinates)
- Generated bounding boxes
- **Assigned the same box to BOTH patches** if rock is in overlap region

**Result:** Ground truth labels have duplicates!

---

## We Need to Remove GT Duplicates

**We will use IoU=0.9**
- Ground truth boxes in overlap regions should be **identical** (same global coordinates)
- IoU=0.9 ensures we only remove exact duplicates, not nearby distinct rocks
- Lower threshold (0.5) might incorrectly merge two real rocks that are close together


## Evaluation Strategy

### **Step 1: Create Clean Ground Truth**

In [1]:
# Remove exact duplicates from ground truth
!python ../scripts/postprocessing/yolo_to_shapefile.py \
  --tif_dir ../data/swisstopo_data/images/test \
  --labels_dir ../data/swisstopo_data/labels/test \
  --out ../outputs/shapefiles/test_GT_with_duplicates.shp \
  --nms_iou 0.0

!python ../scripts/postprocessing/yolo_to_shapefile.py \
  --tif_dir ../data/swisstopo_data/images/test \
  --labels_dir ../data/swisstopo_data/labels/test \
  --out ../outputs/shapefiles/test_GT_clean.shp \
  --nms_iou 0.95  # Very high - only exact duplicates

/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Total detections before NMS: 369
✅ Wrote 369 boxes to ../outputs/shapefiles/test_GT_with_duplicates.shp
CRS: EPSG:2056
/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
Total detections before NMS: 369
✂️  Detections after NMS (IoU=0.95): 361
🗑️  Removed duplicates: 8
✅ Wrote 361 boxes to ../outputs/shapefiles/test_GT_clean.shp
CRS: EPSG:2056


In [5]:
!ls

data  models  notebooks  outputs  README.md  scripts


In [2]:
%cd ..

/home/donia/Switzerland_Expansion/switzerland_nationwide_rock_detection


/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [8]:
!python scripts/analysis/duplicate_suppression/find_duplicates_from_labels.py 

FINDING REAL DUPLICATES FROM YOLO LABELS
Working directory: /home/donia/Switzerland_Expansion/switzerland_nationwide_rock_detection

Found 6 unique tiles
Total patches: 96

Tile 2587_1133: 16 patches
  Found 3 duplicate pair(s):
    2587_1133_0_2.txt ↔ 2587_1133_1_3.txt
      Distance: 5.34 pixels
      Global coords 1: (1423.2, 614.4)
      Global coords 2: (1424.4, 609.2)
    2587_1133_0_2.txt ↔ 2587_1133_1_3.txt
      Distance: 1.70 pixels
      Global coords 1: (1378.4, 448.0)
      Global coords 2: (1379.6, 449.2)
    2587_1133_2_2.txt ↔ 2587_1133_3_3.txt
      Distance: 5.34 pixels
      Global coords 1: (1320.8, 1365.6)
      Global coords 2: (1315.6, 1366.8)

Tile 2595_1130: 16 patches
  No duplicates found

Tile 2611_1139: 16 patches
  Found 2 duplicate pair(s):
    2611_1139_0_0.txt ↔ 2611_1139_1_1.txt
      Distance: 1.70 pixels
      Global coords 1: (454.4, 537.6)
      Global coords 2: (455.6, 538.8)
    2611_1139_0_0.txt ↔ 2611_1139_1_1.txt
      Distance: 1.70 pixels
  

In [9]:
!python scripts/analysis/duplicate_suppression/visualize_duplicate_pairs.py

VISUALIZING REAL DUPLICATE PAIRS
Working directory: /home/donia/Switzerland_Expansion/switzerland_nationwide_rock_detection

Found 8 duplicate pairs to visualize

--- Processing Pair 1/8 ---
  2587_1133_0_2.txt ↔ 2587_1133_1_3.txt
  Loading 2587_1133_0_2.txt...
/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
  Loading 2587_1133_1_3.txt...
  ✅ Saved to outputs/duplicate_visualizations_real/real_duplicate_pair_01.png

--- Processing Pair 2/8 ---
  2587_1133_0_2.txt ↔ 2587_1133_1_3.txt
  Loading 2587_1133_0_2.txt...
/home/donia/.conda/envs/rock_detection/lib/python3.9/site-packages/rasterio/__init__.py:356: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=